In [1]:
import os
import copy
import numpy as np
import laspy as lp
import open3d as o3d

def to_open3d(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    return pcd

def las_to_open3d(las_source_path, target_path):
    las = lp.read(las_source_path)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    pcd = to_open3d(points)
    o3d.io.write_point_cloud(target_path, pcd)

# def draw_registration_result(source, target, transformation):
#     source_temp = copy.deepcopy(source)
#     target_temp = copy.deepcopy(target)
#     source_temp.paint_uniform_color([1, 0.706, 0])
#     target_temp.paint_uniform_color([0, 0.651, 0.929])
#     source_temp.transform(transformation)
#     o3d.visualization.draw_geometries([source_temp, target_temp])

def get_ground_points(las_source_path, salmuera):
    las = lp.read(las_source_path)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    data_points = np.hstack((points,
                        np.expand_dims(las.return_number, -1),
                        np.expand_dims(las.number_of_returns, -1)))
    one_return = data_points[(data_points[:, -1] == 1)][:, :3]
    return one_return[one_return[:,2] > salmuera] 


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
salmueras = {
    'km11': 2300.866,
    'km12': 2300.582,
    'km13': 2300.647,
    'pam2': 2302.185,
    '2a': 2301.129,
    '3a': 2300.978
}

POZA = 'km11'
THRESHOLD = 0.02
SALMUERA = salmueras.get(POZA, None)
TRANS_INIT = np.asarray([[1.0, 0.0, 0.0, 0.0], 
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0], 
                         [0.0, 0.0, 0.0, 1.0]])

In [3]:
source_path = 'data/vuelos_marzos/29032025/con talud/100m_10ms_100khz_plena_luz/km11_sectora_100m_10ms_100khz_plena_luz_0_0.las'
target_path = 'data/vuelos_marzos/29032025/con talud/100m_10ms_100khz_madrugada/km11_sectora_100m_10ms_100khz_madrugada_0_0.las'
pcd_source_path = source_path.replace('.las', '.pcd')
pcd_target_path = target_path.replace('.las', '.pcd')

In [4]:
src_ground_points = get_ground_points(source_path, SALMUERA)
pcd_src_ground_points = to_open3d(src_ground_points)

dst_ground_points = get_ground_points(target_path, SALMUERA)
pcd_dst_ground_points = to_open3d(dst_ground_points)

pcd_src_ground_points.estimate_normals()
pcd_dst_ground_points.estimate_normals()

print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
            pcd_src_ground_points, 
            pcd_dst_ground_points, 
            THRESHOLD, 
            TRANS_INIT,
            o3d.pipelines.registration.TransformationEstimationPointToPlane()
        )

print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)

transformation = reg_p2l.transformation

Apply point-to-plane ICP
RegistrationResult with fitness=1.437873e-03, inlier_rmse=1.527831e-02, and correspondence_set size of 2439
Access transformation to get result.
Transformation is:
[[ 9.99999995e-01  2.78394492e-05  9.25432653e-05 -2.06301841e+02]
 [-2.78337248e-05  9.99999998e-01 -6.18567743e-05  1.58850666e+01]
 [-9.25449871e-05  6.18541982e-05  9.99999994e-01 -4.05631368e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [5]:
# Read existing LAS
las = lp.read(source_path)

src_las_data_points_transformed = np.dot(transformation[:3, :3], las.xyz.T).T + transformation[:3, 3]

# Overwrite coordinates with your transformed point cloud
las.x = src_las_data_points_transformed[:, 0]
las.y = src_las_data_points_transformed[:, 1]
las.z = src_las_data_points_transformed[:, 2]

# Save as a new file (don’t overwrite original unless sure)
output_path = source_path.replace('.las', '_transformed.las')
print(f"Saving transformed LAS to {output_path}")
las.write(output_path)

Saving transformed LAS to data/vuelos_marzos/29032025/con talud/100m_10ms_100khz_plena_luz/km11_sectora_100m_10ms_100khz_plena_luz_0_0_transformed.las
